In [132]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [133]:
%reload_ext autoreload

In [134]:
import json

with open('topic_testing/mixed_playlist.txt', 'rb') as f:
    request = json.load(f)

In [456]:
# for segi in request["body"]["segments"]:
#     if segi["transcriber"]=="aws":
#         print (segi["originalText"])
#         print ("\n\n\n")

In [135]:
from main import handler

res = handler(request, None)

printing ele
{"level": "info", "filename": "scorer.py", "lineno": 26, "module": "scorer", "batches count": 7, "number of sentences": 665, "ts": "2019-10-15T13:39:46.776823Z", "msg": "computing in batches"}
{"level": "info", "filename": "scorer.py", "lineno": 26, "module": "scorer", "batches count": 7, "number of sentences": 665, "ts": "2019-10-15T13:39:46.777557Z", "msg": "computing in batches"}
{"level": "info", "filename": "scorer.py", "lineno": 26, "module": "scorer", "batches count": 7, "number of sentences": 665, "ts": "2019-10-15T13:39:46.786661Z", "msg": "computing in batches"}
{"level": "info", "filename": "scorer.py", "lineno": 26, "module": "scorer", "batches count": 7, "number of sentences": 665, "ts": "2019-10-15T13:39:46.787383Z", "msg": "computing in batches"}
{"level": "info", "filename": "scorer.py", "lineno": 26, "module": "scorer", "batches count": 7, "number of sentences": 665, "ts": "2019-10-15T13:39:46.787999Z", "msg": "computing in batches"}
{"level": "info", "fil

{"level": "info", "filename": "scorer.py", "lineno": 33, "module": "scorer", "iteration count": 2, "ts": "2019-10-15T13:42:04.796414Z", "msg": "Request Sent"}
{"level": "info", "filename": "scorer.py", "lineno": 33, "module": "scorer", "iteration count": 2, "ts": "2019-10-15T13:42:04.797497Z", "msg": "Request Sent"}
{"level": "info", "filename": "scorer.py", "lineno": 33, "module": "scorer", "iteration count": 2, "ts": "2019-10-15T13:42:04.798509Z", "msg": "Request Sent"}
{"level": "info", "filename": "scorer.py", "lineno": 33, "module": "scorer", "iteration count": 2, "ts": "2019-10-15T13:42:04.799830Z", "msg": "Request Sent"}
{"level": "info", "filename": "scorer.py", "lineno": 33, "module": "scorer", "iteration count": 2, "ts": "2019-10-15T13:42:04.801318Z", "msg": "Request Sent"}
{"level": "info", "filename": "scorer.py", "lineno": 33, "module": "scorer", "iteration count": 2, "ts": "2019-10-15T13:42:04.802489Z", "msg": "Request Sent"}
Recieved Response. Checking if it's valid.
Val

{"level": "info", "filename": "scorer.py", "lineno": 31, "module": "scorer", "iteration count:": 5, "ts": "2019-10-15T13:43:07.506167Z", "msg": "getting feature vector from mind service"}
{"level": "info", "filename": "scorer.py", "lineno": 31, "module": "scorer", "iteration count:": 5, "ts": "2019-10-15T13:43:07.506801Z", "msg": "getting feature vector from mind service"}
{"level": "info", "filename": "scorer.py", "lineno": 31, "module": "scorer", "iteration count:": 5, "ts": "2019-10-15T13:43:07.507483Z", "msg": "getting feature vector from mind service"}
{"level": "info", "filename": "scorer.py", "lineno": 31, "module": "scorer", "iteration count:": 5, "ts": "2019-10-15T13:43:07.508179Z", "msg": "getting feature vector from mind service"}
{"level": "info", "filename": "scorer.py", "lineno": 33, "module": "scorer", "iteration count": 5, "ts": "2019-10-15T13:43:34.990789Z", "msg": "Request Sent"}
{"level": "info", "filename": "scorer.py", "lineno": 33, "module": "scorer", "iteration c

{"level": "info", "filename": "grouper_segments.py", "lineno": 131, "module": "grouper_segments", "outlier threshold is : ": 0.7122736840836394, "ts": "2019-10-15T13:44:45.590285Z", "msg": "Outlier Score"}
{"level": "info", "filename": "grouper_segments.py", "lineno": 131, "module": "grouper_segments", "outlier threshold is : ": 0.7122736840836394, "ts": "2019-10-15T13:44:45.590880Z", "msg": "Outlier Score"}
{"level": "info", "filename": "grouper_segments.py", "lineno": 131, "module": "grouper_segments", "outlier threshold is : ": 0.7122736840836394, "ts": "2019-10-15T13:44:45.591460Z", "msg": "Outlier Score"}
{"level": "info", "filename": "grouper_segments.py", "lineno": 131, "module": "grouper_segments", "outlier threshold is : ": 0.7122736840836394, "ts": "2019-10-15T13:44:45.592150Z", "msg": "Outlier Score"}
{"level": "info", "filename": "grouper_segments.py", "lineno": 131, "module": "grouper_segments", "outlier threshold is : ": 0.7122736840836394, "ts": "2019-10-15T13:44:45.5927

('But I see you want to do is log into your account and click on email campaign and then create campaign here.', '2019-10-15T07:07:06Z', '2cd90f0674f348cc922acd6b8782ba0f', '061deb77473f4fa2863516df62738d4c')
('Enter my sender name, then choose my singer and reply to email addresses and click Create content here.', '2019-10-15T07:07:06Z', '2cd90f0674f348cc922acd6b8782ba0f', '061deb77473f4fa2863516df62738d4c')
('I can choose what type of email I want to send with templates.', '2019-10-15T07:07:06Z', '2cd90f0674f348cc922acd6b8782ba0f', '061deb77473f4fa2863516df62738d4c')
('I can use layouts already available in SoHo and then add my own content.', '2019-10-15T07:07:06Z', '2cd90f0674f348cc922acd6b8782ba0f', '061deb77473f4fa2863516df62738d4c')
('I can use imports, upload templates from my computer.', '2019-10-15T07:07:06Z', '2cd90f0674f348cc922acd6b8782ba0f', '061deb77473f4fa2863516df62738d4c')
('I can create a custom HTML template from scratch, or I can upload templates from Google docks.'

...

{"level": "info", "filename": "grouper_segments.py", "lineno": 485, "module": "grouper_segments", "PIMs": {"0": {"segment0": [["I'm sure you have. Have you ever been to? I don't know about a text. A crew goes to further different experiences the truth of my visions and your experiences. It's highly likely there was a very sophisticated and about her visiting pa*ses behind it. It takes a team of people, goes from a creative process called user experience designer. So that was your experience design. Anyways. It was about what, When, where, Why, how and who would use this to product once we have understood that n**es and once a user and we could design an awesome solution for them. But you don't throw the road. Usually there are many different kinds of creative people collaborating together. It's like a Jan your party, right ISS. This creative people bring in route skills to the table. Let's take your smartphone, for example. So for speaking, the returns very obvious areas of design Here

{"level": "info", "filename": "grouper_segments.py", "lineno": 485, "module": "grouper_segments", "PIMs": {"0": {"segment0": [["I'm sure you have. Have you ever been to? I don't know about a text. A crew goes to further different experiences the truth of my visions and your experiences. It's highly likely there was a very sophisticated and about her visiting pa*ses behind it. It takes a team of people, goes from a creative process called user experience designer. So that was your experience design. Anyways. It was about what, When, where, Why, how and who would use this to product once we have understood that n**es and once a user and we could design an awesome solution for them. But you don't throw the road. Usually there are many different kinds of creative people collaborating together. It's like a Jan your party, right ISS. This creative people bring in route skills to the table. Let's take your smartphone, for example. So for speaking, the returns very obvious areas of design Here

{"level": "info", "filename": "grouper_segments.py", "lineno": 485, "module": "grouper_segments", "PIMs": {"0": {"segment0": [["I'm sure you have. Have you ever been to? I don't know about a text. A crew goes to further different experiences the truth of my visions and your experiences. It's highly likely there was a very sophisticated and about her visiting pa*ses behind it. It takes a team of people, goes from a creative process called user experience designer. So that was your experience design. Anyways. It was about what, When, where, Why, how and who would use this to product once we have understood that n**es and once a user and we could design an awesome solution for them. But you don't throw the road. Usually there are many different kinds of creative people collaborating together. It's like a Jan your party, right ISS. This creative people bring in route skills to the table. Let's take your smartphone, for example. So for speaking, the returns very obvious areas of design Here

{"level": "info", "filename": "grouper_segments.py", "lineno": 485, "module": "grouper_segments", "PIMs": {"0": {"segment0": [["I'm sure you have. Have you ever been to? I don't know about a text. A crew goes to further different experiences the truth of my visions and your experiences. It's highly likely there was a very sophisticated and about her visiting pa*ses behind it. It takes a team of people, goes from a creative process called user experience designer. So that was your experience design. Anyways. It was about what, When, where, Why, how and who would use this to product once we have understood that n**es and once a user and we could design an awesome solution for them. But you don't throw the road. Usually there are many different kinds of creative people collaborating together. It's like a Jan your party, right ISS. This creative people bring in route skills to the table. Let's take your smartphone, for example. So for speaking, the returns very obvious areas of design Here

{"level": "info", "filename": "grouper_segments.py", "lineno": 485, "module": "grouper_segments", "PIMs": {"0": {"segment0": [["I'm sure you have. Have you ever been to? I don't know about a text. A crew goes to further different experiences the truth of my visions and your experiences. It's highly likely there was a very sophisticated and about her visiting pa*ses behind it. It takes a team of people, goes from a creative process called user experience designer. So that was your experience design. Anyways. It was about what, When, where, Why, how and who would use this to product once we have understood that n**es and once a user and we could design an awesome solution for them. But you don't throw the road. Usually there are many different kinds of creative people collaborating together. It's like a Jan your party, right ISS. This creative people bring in route skills to the table. Let's take your smartphone, for example. So for speaking, the returns very obvious areas of design Here

In [136]:
group = json.loads(res['body'])

In [137]:
for g in group['group'].keys():
    if len(group['group'][g])>1:
        print (g)

0
8
25
30
31
32
41
47
57
60
62
82
86
96
100
102
115
119
124
129
132
136
138
142
146
151
156
168


In [71]:
# json.dumps(group)
# with open("result_testing.json", "w") as f:
#     f.write(json.dumps(group))

In [138]:
import iso8601
from datetime import datetime
# meeting start time.
def formatTime(tz_time, datetime_object=False):
    isoTime = iso8601.parse_date(tz_time)
    ts = isoTime.timestamp()
    ts = datetime.utcfromtimestamp(ts).strftime("%Y-%m-%d %H:%M:%S:%f")

    if datetime_object:
        ts = datetime.fromisoformat(ts)
    return ts

#m_time = formatTime("2019-09-19T06:05:00Z", True) #eng_19
#m_time = formatTime("2019-09-20T07:12:00Z", True) #eng_front_end_20
#m_time = formatTime("2019-09-24T06:11:00Z", True) #eng_24
#m_time = formatTime("2019-10-04T05:44:00Z", True)  #podcast_04
#m_time = formatTime("2019-10-08T11:55:00Z", True)  #podcast_08
m_time = formatTime("2019-10-14T06:04:00Z", True)
#m_time = formatTime("2019-09-16T09:53:21Z", True)
#m_time = formatTime("2019-07-04T12:15:14Z", True)
for i in group['group'].keys():
    print ("\n\n\nPIMs ", i)
    print ("\n\nDiscussion:\n\n ")
    for seg in group['group'][i]:
        print ("Minutes from the start of the meeting: ", formatTime(seg['startTime'], True) - m_time , seg['id'],"\n")
        print (seg['originalText'],"\n")





PIMs  0


Discussion:

 
Minutes from the start of the meeting:  1 day, 0:19:53 a41c9932ef794c799dcf14f1564e4acc 

I'm sure you have. Have you ever been to? I don't know about a text. A crew goes to further different experiences the truth of my visions and your experiences. It's highly likely there was a very sophisticated and about her visiting pa*ses behind it. It takes a team of people, goes from a creative process called user experience designer. So that was your experience design. Anyways. It was about what, When, where, Why, how and who would use this to product once we have understood that n**es and once a user and we could design an awesome solution for them. But you don't throw the road. Usually there are many different kinds of creative people collaborating together. It's like a Jan your party, right ISS. This creative people bring in route skills to the table. Let's take your smartphone, for example. So for speaking, the returns very obvious areas of design Here. One is a

In [26]:
import iso8601
from datetime import datetime
# meeting start time.
def formatTime(tz_time, datetime_object=False):
    isoTime = iso8601.parse_date(tz_time)
    ts = isoTime.timestamp()
    ts = datetime.utcfromtimestamp(ts).strftime("%Y-%m-%d %H:%M:%S:%f")

    if datetime_object:
        ts = datetime.fromisoformat(ts)
    return ts

#m_time = formatTime("2019-09-19T06:05:00Z", True) #eng_19
#m_time = formatTime("2019-09-20T07:12:00Z", True) #eng_front_end_20
#m_time = formatTime("2019-09-24T06:11:00Z", True) #eng_24
#m_time = formatTime("2019-10-04T05:44:00Z", True) #podcast_04
m_time = formatTime("2019-10-08T11:55:00Z", True) #podcast_08
#m_time = formatTime("2019-09-22T09:37:00Z", True)
#m_time = formatTime("2019-09-16T09:53:21Z", True)
#m_time = formatTime("2019-07-04T12:15:14Z", True)
for i in group['group'].keys():
    print ("\n\n\nPIMs ", i)
    print ("\n\nDiscussion:\n\n ")
    for seg in group['group'][i]:
        print ("Minutes from the start of the meeting: ", formatTime(seg['startTime'], True) - m_time , seg['id'],"\n")
        print (seg['originalText'],"\n")





PIMs  3


Discussion:

 
Minutes from the start of the meeting:  0:11:47 06d4cfa410ee47f6ba66ad53dbe41ccf 

never, never, never develops any product. There are certain issues which are probably almost every time. Markers. Well, that problem is whenever you're designing a project, it was absolutely fine in your machine, the developer machine. But as soon as that product is being moved onto the production state, maybe onto servers or maybe somebody's else computer. Maybe your friends are maybe a manager. This computer. In that case, the product usually fails to work with the same performance or same optimization or the same level of working, even when that product has moved from one place to another place. Although here we're talking about the big skill products that include something like no django myfriend Javascript in all those stuffs just to give you a bare minimum. Basic example, whether you know of any bets site using PHP or maybe sp dot net. Whenever you work on that project a

In [431]:
i = 1
j = 1
if i !=1 or j !=1:
    print (True)

In [277]:
import sys
sys.path.append("../../../ai-engine/pkg/")

from graphrank.core import GraphRank
from graphrank.utils import GraphUtils, TextPreprocess

gr = GraphRank()
tp = TextPreprocess()
gu = GraphUtils()

def get_desc(sentence):
    original_tokens, pos_tuple, filtered_pos_tuple = tp.preprocess_text(sentence, filter_by_pos=True, stop_words=False)
    word_graph = gr.build_word_graph(graph_obj=None, input_pos_text=pos_tuple, window=4, preserve_common_words=False)
    normal_keyphrase = gr.get_keyphrases(word_graph, pos_tuple, post_process=True)
    desc_keyphrase = gr.get_keyphrases(word_graph, pos_tuple, descriptive=True, post_process_descriptive=True)
    desc_keyphrase = sorted(desc_keyphrase, key=lambda kv:kv[1], reverse=True)
    normal_kp = [phrase for phrase, score in normal_keyphrase]
    desc_kp = [phrase for phrase, score in desc_keyphrase]
    
    return normal_kp, desc_kp

In [278]:
user_id_map = {}
user_id_map = {"84fbaa66a2474ea29ae053f3a2e519d6":"Mithun","75bdf310110b4b8fab88b16fafce920e":"Trishanth Diwate","b1e8787a9a1f4859ac11cbb6a8124fd9": "Venkata Dikshit", "fb52cb663aec4795aee38ccfd904d315":"Reagan Rewop", "81a3e15469374fceba1cf972faa209b2":"Arjun Kini", "ecfeeb757f0a4d47af1ebd513929264a":"Shubham", "62b6ae1d7f834b0bb2055f7c72bc3368":"Karthik Muralidharan", "1a21542584494fcaba957d768b595b80":"Vamshi Krishna", "7e7ccbba232d411aa95ad3f244a35f40":"Shashank", "65bb83952fb54409a4bb59bb707f1375":"Vani", "0bbbfe84c66145af8d0ffcd5258bba38":"Parshwa Nemi Jain"}

for groupid in group["group"]:
    user_list =[]
    seg_list = []
    keyphrase = []
    if len(group["group"][groupid])>1:
        for segi in group["group"][groupid]:
            if segi['spokenBy'] not in user_list:
                user_list.append(segi['spokenBy'])
            seg_list.append(segi['originalText'])
            #keyphrase.append(get_desc(segi['originalText']))
        #print ("User", end=" ")
        print (*[user_id_map[user] for user in user_list], sep=", ", end=" ")
        print ("Discussed \n\n Text: ", *seg_list, "\n\n")
        print (*get_desc(" ".join(sent for sent in seg_list))[1][:5], sep=", ")
        print ( "\n\n\n\n")

Karthik Muralidharan, Venkata Dikshit Discussed 

 Text:  Also the depends on how based on user. At least for now yeah, but it's new to will figure out that I think eventually once once you guys figure out on on the graph database looking if we have around that I think I don't see any problem in screen. So let's. 


guys figure, graph database, problem in screen, user





Karthik Muralidharan, Mithun, Parshwa Nemi Jain Discussed 

 Text:  Thing is we discussed with like earlier and we are thinking of the feature. The extraction service for topic extraction service that we have and is like so that is the old one so we need remove all that changes to the call so can you make a change like in for example, wherever will call that service right now I think happens many that we should in slack right call use three servers users and then remove from platform and day also. But I think there are two places I I remember one is marker registered suggested marker topic extraction and we are there

help us with building vocabulary of, future for maybe topic extraction, fifty six maximum of work, call to client, issues will desist Chime





Venkata Dikshit, Karthik Muralidharan Discussed 

 Text:  And then then these two can be you know team so for the. New structure for the minds that you're thinking storing it as a file and. If you finalize on the I think most likely is going to be graph right. Okay so if you finalize on the graph data database we i think you have I think you eventually because at least it's not really if you if you go. Let figure. So if start, it's not beyond like it's like a five thousand node graph it could so so if it's easy to use it as the file system as a file I think or maybe that's not scalable, but that's a graph if there all stuff we they don't they which may not. 


finalize on the graph data database, node graph, start, file system, structure for the minds





Shashank, Karthik Muralidharan Discussed 

 Text:  Tries for so it works fine so do have

In [173]:
groups = []
for i in group['group'].keys():
    if len(group['group'][i])==1:
        continue
    else:
        temp = []
        for seg in group['group'][i]:
            temp.append(seg['originalText'])
        groups.append(' '.join(temp))

In [ ]:
import pandas as pd
df = pd.read_csv("../data/processed/master_tag+para_cluster_df.csv")
import ast
tag = [] 
for tags in df['tags']:
    for t in ast.literal_eval(tags):
        tag.append(t)
print ("no of tags present:", len(tag))
unique_tag_unr = list(set(tag))
unique_tag = []
for u in unique_tag_unr:
    if tag.count(u) > 10:
        unique_tag.append(u)
print ("no of unique tags present:", len(unique_tag))

In [ ]:
import boto3
from boto3 import client
from botocore.client import Config
import numpy as np
aws_config = Config(
        connect_timeout=60,
        read_timeout=300,
        retries={"max_attempts": 0},
        region_name="us-east-1",
    )
lambda_client = client("lambda", config=aws_config)

def get_embeddings(input_list, req_data=None):

    if req_data is None:
        lambda_payload = {"body": {"text_input": input_list}}
    else:
        lambda_payload = {"body": {"request": req_data, "text_input": input_list}}

    try:
        #logger.info("Invoking lambda function")
        invoke_response = lambda_client.invoke(
            FunctionName="keyphrase_ranker",
            InvocationType="RequestResponse",
            Payload=json.dumps(lambda_payload),
        )

        lambda_output = (
            invoke_response["Payload"].read().decode("utf8").replace("'", '"')
        )
        response = json.loads(lambda_output)
        status_code = response["statusCode"]
        response_body = response["body"]

        if status_code == 200:
            embedding_vector = np.asarray(json.loads(response_body)["embeddings"])

        else:
            embedding_vector = np.asarray(json.loads(response_body)["embeddings"])
    except Exception as e:
        print (e)
        pass
    return embedding_vector

In [ ]:
fv = get_embeddings(groups, req_data=None)
fv_tag = get_embeddings([i for i in unique_tag if i!=""], req_data=None)

In [ ]:
from scipy.spatial.distance import cosine
for index, tex in enumerate(groups):
    closest = -1
    closest_tag = None
    closest_2= -1
    closest_tag_2 = None
    closest_3 = -1
    closest_tag_3 = None
    closest_4= -1
    closest_tag_4 = None
    for index2, t in enumerate([i for i in unique_tag if i!=""]):
        if t!="":
            score = 1 - cosine(fv[index],fv_tag[index2])
            if score > closest:
                closest = score
                closest_tag = t
            elif score > closest_2:
                closest_tag_2 = t
                closest_2 = score
            elif score > closest_3:
                closest_tag_3 = t
                closest_3 = score
            elif score > closest_4:
                closest_tag_4 = t
                closest_4 = score
    print ("\n\n\nsentence: \n\n", tex)
    print ("\n most similar tags: ", str(closest_tag) + " , " + str(closest_tag_2) + " , " + str(closest_tag_3) + " , " + str(closest_tag_4) )